# Better Curbs Project: Curb Performance Metric Calculator

### This notebook can be used to generate Curb Productivity Index (CPIx) and Curb Accessibility (CAx) metrics for a single simulation run.

#### Developed by Tom Maxner, University of Washington
#### Last updated: 12/11/2022

### The sequence of operations in this notebook are as follows:
#### 1. Load script for .fzp (vehicle trajectory) file parsing tool, and modules for analyzing data;
#### 2. Load and parse select .fzp file;
#### 3. Preprocess and filter data;
#### 4. Calculate curb performance metrics;
        4.4. Curb Productivity for each blockface: calculated for goods and passengers, & for each vehicle type.
        4.5. Curb Accessibility for each blockface: calculated in total, for goods and passengers, & for each vehicle type.
#### 5. Calculate parking space occupancy;
        Calculated for each space by vehicle type and averages for entire blockfaces.
#### 6. Export processed data to .csv for further analysis.

### NOTE: User must update directory and file names in Steps 2 and 6. It may also be necessary for users to update variables defined in Step 2 and space numbers in Steps 5 and 6 to match their own inputs.

## Step 1. Load script for .fzp file parsing tool, and modules for analyzing data.

In [1]:
# Set display settings (optional)
# cells will fill entire width of the browser
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

#Tells Jupyter to reload custom classes from scratch everytime an import cell is run, if you edit a custom class
#between imports Jupyter would otherwise need to be restarted completely. NOTE: old class objects in the 
#current namespace will cause errors at execution
%load_ext autoreload
%autoreload 2

#switches matplotlib to show plots in the browser rather than opening a new window
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

#passive library that generates nicer looking plots
import seaborn
seaborn.set()

In [3]:
def pandas_parse_fzp_vissim_vehicle_records(filename):
    #this function parses a vehicle records fzp file and 
    #reads the data into a pandas dataframe
    
    #FZP file notes
    #lines beginning with * are comments
    #escape character \ is not an escape character
    #line beginning with $ is header
    #data is ; delimited
    #dataframe will come back with all strings as data

    data = []
    
    with open(filename, 'r') as d:
        for line in d:
            if line[0] == "*":
                pass
            elif line[0:8] == "$VEHICLE": #column header doesn't precisely correspond to columns
                header = line.replace("\\","_").strip().split(";") #VISSIM uses escape characters in places
                header[0] = header[0].split(":")[1] #getting rid of garbage at start of column header
            elif line[0] == "$":
                pass
            else:       
                data.append(line.strip().split(";"))
                
    out_frame = pd.DataFrame(data)
    out_frame = out_frame.drop(labels=0, axis=0) #ends up reading first line anyway because of unicode
    out_frame.columns = header
    
    return(out_frame)

## Step 2. Load and parse select .fzp file.

In [4]:
datapath = "D:\\VISSIM_Sims_05_13_2_Lane\\Scen1" #path to the location of the FZP file you want to read / Directory
filename = "Seattle_Atomic_Network_Scen1_experiment_121_001.fzp" #FZP file you want to read

SCEN = filename[27:28]
EX = filename[40:-8]

S01_02 = pandas_parse_fzp_vissim_vehicle_records(os.path.join(datapath, filename)) # .fzp parsing tool. S01_02 is the dataframe that will be used throughout.

In [10]:
# Define some of the variables to be used later

# vehicles that unload/load passengers = '100'=personal vehicles, '640'=TNC or PUDO vehicles, '300'=buses
# vehicles that unload/load goods = '650'=cargo vans / work trucks, '200'=heavy goods vehicles

Sp1 = 12 # the number of available parking spaces on South side of study area; includes bus stop (2 usable spaces)
Sp2 = 11 # the number of available parking spaces on North side of study area; includes bus stop (2 usable spaces)
t = 8999 # the length of simulation (in seconds)
th = 2.5 #(simulation time in hours)


## Step 3. Preprocess and filter data.

In [5]:
# First separate the "Parked" and "Failed to Park" records
# Create filters for vehicles that parked and those that failed to park
Park_Filter_S01_02 = (S01_02['PARKSTATE'] == 'Parked')
FailedPark_Filter_S01_02 = (S01_02['PARKSTATE'] == 'Parking request declined')

# Create dataframes 
Park_S01_02 = S01_02[Park_Filter_S01_02]
Fail_S01_02 = S01_02[FailedPark_Filter_S01_02]

In [7]:
# Get every unique vehicle entry for parked vehicles and those that fail to park.
Park_S01_02 = Park_S01_02.drop_duplicates(subset = ["NO"])
Fail_S01_02 = Fail_S01_02.drop_duplicates(subset = ["NO"])

#Park_S01_02
#Fail_S01_02

In [9]:
# Convert object variables to integer
Park_S01_02["DWELLTM"] = Park_S01_02["DWELLTM"].astype(str).astype(float)
Park_S01_02["VEHTYPE"] = Park_S01_02["VEHTYPE"].astype(str).astype(float)
Fail_S01_02["DWELLTM"] = Fail_S01_02["DWELLTM"].astype(str).astype(float)
Fail_S01_02["VEHTYPE"] = Fail_S01_02["VEHTYPE"].astype(str).astype(float)
Park_S01_02["OCCUP"] = Park_S01_02["OCCUP"].astype(str).astype(float)
Fail_S01_02["OCCUP"] = Fail_S01_02["OCCUP"].astype(str).astype(float)
Park_S01_02["DESTPARKLOT"] = Park_S01_02["DESTPARKLOT"].astype(str).astype(float)

# Convert variables related to bus analysis to integer
S01_02["LANE_LINK_NO"] = S01_02["LANE_LINK_NO"].astype(str).astype(float)
S01_02["LANE_INDEX"] = S01_02["LANE_INDEX"].astype(str).astype(float)     
S01_02["OCCUP"] = S01_02["OCCUP"].astype(str).astype(float)                   
S01_02["DWELLTM"] = S01_02["DWELLTM"].astype(str).astype(float)                

#print(Park_S01_02.dtypes)

## Step 4. Calculate curb performance metrics.

### 4.1. Add bus passengers to variables

In [11]:
# Filter original df (S01_02) for all bus records
Bus_Filter_S01_02 = (S01_02['VEHTYPE'] == '300')
Bus_S01_02 = S01_02[Bus_Filter_S01_02]
#Bus_S01_02

In [13]:
# Filter bus df for only records tied to blocks with parking AND if buses entered parking lane (Lane 1).
Bus_Link_Filter = (((Bus_Bus_S01_02['LANE_LINK_NO'] == 1) | (Bus_Bus_S01_02['LANE_LINK_NO'] == 2) \
                  & (Bus_Bus_S01_02['LANE_INDEX'] == 1))
Bus_Lane = Bus_Bus_S01_02[Bus_Link_Filter]
#Bus_Lane_S01_02

In [14]:
# Drop duplicate bus records - Assumes all of these buses stop, even though I don't think they are the correct bus records (orange and purple lines)
Bus_Lane_S01_02 = Bus_Lane_S01_02.drop_duplicates(subset = ["NO"])
#Bus_Lane_S01_02.head(50)

In [15]:
# Assign passengers to buses entering bus stops. VISSIM does have the ability to output this info, but it was not included in the base runs.
Bus1 = Bus_Lane_S01_02.loc[Bus_Lane_S01_02['LANE_LINK_NO']==1, 'LANE_INDEX'].sum()
Bus2 = Bus_Lane_S01_02.loc[Bus_Lane_S01_02['LANE_LINK_NO']==2, 'LANE_INDEX'].sum()
Bus_Pax1 = round(Bus1 * np.random.normal(10.0, 1.0))
Bus_Pax2 = round(Bus2 * np.random.normal(10.0, 1.0))

print(Bus1, Bus2, Bus_Pax1, Bus_Pax2)

15.0 12.0 151.0 102.0


### 4.2. Add passenger and goods counts to "Park..." and "Fail..." dataframes

In [16]:
# add passenger count and package columns
conditionspax = [
    (Park_S01_02['VEHTYPE'] == 100) & (Park_S01_02['DWELLTM'] < 600),
    (Park_S01_02['VEHTYPE'] == 100) & (Park_S01_02['DWELLTM'] > 600),
    (Park_S01_02['VEHTYPE'] == 640) & (Park_S01_02['DWELLTM'] < 600),
    (Park_S01_02['VEHTYPE'] == 640) & (Park_S01_02['DWELLTM'] > 600),
    (Park_S01_02['VEHTYPE'] == 300) & (Park_S01_02['DWELLTM'] < 300),
    (Park_S01_02['VEHTYPE'] == 300) & (Park_S01_02['DWELLTM'] > 300),
    (Park_S01_02['VEHTYPE'] == 650),
    (Park_S01_02['VEHTYPE'] == 200)
]
choicespax = [1,2,1,2,10,15,0,0] # see publication for justification of pax and goods loaded/unloaded.
Park_S01_02['pax'] = np.select(conditionspax, choicespax, default=0)

conditionsgoods = [
    (Park_S01_02['VEHTYPE'] == 650) & (Park_S01_02['DWELLTM'] < 1800),
    (Park_S01_02['VEHTYPE'] == 650) & (Park_S01_02['DWELLTM'] > 1800),
    (Park_S01_02['VEHTYPE'] == 200) & (Park_S01_02['DWELLTM'] < 1800),
    (Park_S01_02['VEHTYPE'] == 200) & (Park_S01_02['DWELLTM'] > 1800)
]
choicesgoods = [5,10,5,10] # see publication for justification of pax and goods loaded/unloaded.
Park_S01_02['parcels'] = np.select(conditionsgoods, choicesgoods, default=0)

#Park_S01_02.head(5)

In [17]:
# Create separate dataframes for Block face 1 and Block face 2
# Using the 'Lane_Link_No' means you do not need to consider the destination parking lot number (those are not included in fail parking events +
# in scenario 6, the through-lane and bus-stop parking lots are out of sequence)

# Parking dataframe
FacePark1_S01_02_filter = (Park_S01_02['LANE_LINK_NO'] == '1')
FacePark2_S01_02_filter = (Park_S01_02['LANE_LINK_NO'] == '2')
Park1_S01_02 = Park_S01_02[FacePark1_S01_02_filter]
Park2_S01_02 = Park_S01_02[FacePark2_S01_02_filter]

# Failed Park Dataframe
FaceFail1_S01_02_filter = (Fail_S01_02['LANE_LINK_NO'] == '1')
FaceFail2_S01_02_filter = (Fail_S01_02['LANE_LINK_NO'] == '2')
Fail1_S01_02 = Fail_S01_02[FaceFail1_S01_02_filter]
Fail2_S01_02 = Fail_S01_02[FaceFail2_S01_02_filter]

In [18]:
# Defining the variables for curb productivity

#v_ip = vehicles successfully unloading passengers
# vehicles that unload/load passengers = '100', '640'
v_ip1 = Park1_S01_02.loc[Park1_S01_02['VEHTYPE']==100, 'OCCUP'].count()+Park1_S01_02.loc[Park1_S01_02['VEHTYPE']==640, 'OCCUP'].count()+Bus1      ##added bus here
v_ip2 = Park2_S01_02.loc[Park2_S01_02['VEHTYPE']==100, 'OCCUP'].count()+Park2_S01_02.loc[Park2_S01_02['VEHTYPE']==640, 'OCCUP'].count()+Bus2      ##added bus here
v_ip1_TNC = Park1_S01_02.loc[Park1_S01_02['VEHTYPE']==640, 'OCCUP'].count()
v_ip2_TNC = Park2_S01_02.loc[Park2_S01_02['VEHTYPE']==640, 'OCCUP'].count()
v_ip1_Cars = Park1_S01_02.loc[Park1_S01_02['VEHTYPE']==100, 'OCCUP'].count()
v_ip2_Cars = Park2_S01_02.loc[Park2_S01_02['VEHTYPE']==100, 'OCCUP'].count()

#v_ig = vehicles successfully unloading goods
# vehicles that unload/load goods = '650', '200'
v_ig1 = Park1_S01_02.loc[Park1_S01_02['VEHTYPE']==650, 'OCCUP'].count()+Park1_S01_02.loc[Park1_S01_02['VEHTYPE']==200, 'OCCUP'].count()
v_ig2 = Park2_S01_02.loc[Park2_S01_02['VEHTYPE']==650, 'OCCUP'].count()+Park2_S01_02.loc[Park2_S01_02['VEHTYPE']==200, 'OCCUP'].count()

#fv_ip = failed parking attempts - passengers
# vehicles that unload/load passengers = '100', '640'
fv_ip1 = Fail1_S01_02.loc[Fail1_S01_02['VEHTYPE']==100, 'OCCUP'].count()+Fail1_S01_02.loc[Fail1_S01_02['VEHTYPE']==640, 'OCCUP'].count()
fv_ip2 = Fail2_S01_02.loc[Fail2_S01_02['VEHTYPE']==100, 'OCCUP'].count()+Fail2_S01_02.loc[Fail2_S01_02['VEHTYPE']==640, 'OCCUP'].count()
fv_ip1_TNC = Fail1_S01_02.loc[Fail1_S01_02['VEHTYPE']==640, 'OCCUP'].count()
fv_ip2_TNC = Fail2_S01_02.loc[Fail2_S01_02['VEHTYPE']==640, 'OCCUP'].count()
fv_ip1_Cars = Fail1_S01_02.loc[Fail1_S01_02['VEHTYPE']==100, 'OCCUP'].count()
fv_ip2_Cars = Fail2_S01_02.loc[Fail2_S01_02['VEHTYPE']==100, 'OCCUP'].count()

#fv_ig = failed parking attempts - goods
# vehicles that unload/load goods = '650', '200'
fv_ig1 = Fail1_S01_02.loc[Fail1_S01_02['VEHTYPE']==650, 'OCCUP'].count()+Fail1_S01_02.loc[Fail1_S01_02['VEHTYPE']==200, 'OCCUP'].count()
fv_ig2 = Fail2_S01_02.loc[Fail2_S01_02['VEHTYPE']==650, 'OCCUP'].count()+Fail2_S01_02.loc[Fail2_S01_02['VEHTYPE']==200, 'OCCUP'].count()

#pi = total pax served
p_i1 = Park1_S01_02.loc[Park1_S01_02['VEHTYPE']==100, 'OCCUP'].sum()+Park1_S01_02.loc[Park1_S01_02['VEHTYPE']==640, 'OCCUP'].sum()+Bus_Pax1      ##added bus here
p_i2 = Park2_S01_02.loc[Park2_S01_02['VEHTYPE']==100, 'OCCUP'].sum()+Park2_S01_02.loc[Park2_S01_02['VEHTYPE']==640, 'OCCUP'].sum()+Bus_Pax2      ##added bus here. Don't do this for scenario 6.
p_i1_TNC = Park1_S01_02.loc[Park1_S01_02['VEHTYPE']==640, 'OCCUP'].sum()
p_i2_TNC =Park2_S01_02.loc[Park2_S01_02['VEHTYPE']==640, 'OCCUP'].sum()
p_i1_Cars = Park1_S01_02.loc[Park1_S01_02['VEHTYPE']==100, 'OCCUP'].sum()
p_i2_Cars = Park2_S01_02.loc[Park2_S01_02['VEHTYPE']==100, 'OCCUP'].sum()

#gi = total parcels delivered
g_i1 = Park1_S01_02.loc[Park1_S01_02['VEHTYPE']==650, 'parcels'].sum()+Park1_S01_02.loc[Park1_S01_02['VEHTYPE']==200, 'parcels'].sum()
g_i2 = Park2_S01_02.loc[Park2_S01_02['VEHTYPE']==650, 'parcels'].sum()+Park2_S01_02.loc[Park2_S01_02['VEHTYPE']==200, 'parcels'].sum()

In [19]:
# This is a test table to make sure results make sense.
# Variables = {'Variables': ["p_i1", "v_ip1", "g_i1", "v_ip1", "fv_ip1", "fv_ig1", "p_i2", "v_ip2", "g_i1", "v_ig2", "fv_ip2", "fv_ig2"], 
             # 'Description': ["pax loaded/unloaded", "veh unloading pax", "goods loaded/unloaded", "veh unloading goods", "failed pax parking attempts", "failed goods parking attempts", "pax loaded/unloaded", "veh unloading pax", "goods loaded/unloaded", "veh unloading goods", "failed pax parking attempts", "failed goods parking attempts"],
             # 'Values': [p_i1, v_ip1, g_i1, v_ig1, fv_ip1, fv_ig1, p_i2, v_ip2, g_i2, v_ig2, fv_ip2, fv_ig2]}
# Variables_df = pd.DataFrame(data=Variables)
# Variables_df

### 4.4. Calculate Curb Productivity Indices

In [20]:
# Passenger Curb Productivity Index:
CPI_p1 = (((p_i1))/th/Sp1) # Passenger curb productivity on blockface 1
CPI_p2 = (((p_i2))/th/Sp2) # Passenger curb productivity on blockface 2

CPI_TNC_p1 = (((p_i1_TNC))/th/Sp1) # TNC / PUDO curb productivity on blockface 1
CPI_TNC_p2 = (((p_i2_TNC))/th/Sp2) # TNC / PUDO curb productivity on blockface 2
CPI_Cars_p1 = (((p_i1_Cars))/th/Sp1) # Personal vehicle curb productivity on blockface 1
CPI_Cars_p2 = (((p_i2_Cars))/th/Sp2) # Personal vehicle curb productivity on blockface 2
CPI_Bus_p1 = (((Bus_Pax1))/th/Sp1) # Bus curb productivity on blockface 1
CPI_Bus_p2 = (((Bus_Pax2))/th/Sp2) # Bus curb productivity on blockface 2

# Goods curb Productivity Index:
CPI_g1 = (((g_i1))/th/Sp1) # Goods curb productivity on blockface 1
CPI_g2 = (((g_i2))/th/Sp2) # Goods curb productivity on blockface 2

print(CPI_p1, CPI_p2)
print(CPI_TNC_p1, CPI_TNC_p2)
print(CPI_Cars_p1, CPI_Cars_p2)
print(CPI_Bus_p1, CPI_Bus_p2)
print(CPI_g1, CPI_g2)

8.033333333333333 7.0181818181818185
1.0666666666666667 1.309090909090909
1.9333333333333333 2.0
5.033333333333333 3.709090909090909
0.6666666666666666 0.7272727272727273


### 4.5. Calculate Curb Accessibility

In [21]:
# Passenger Curb Accessibility
CAp1 = 1-(fv_ip1 / (fv_ip1 + v_ip1)) # Passenger curb accessibility on blockface 1
CAp2 = 1-(fv_ip2 / (fv_ip2 + v_ip2)) # Passenger curb accessibility on blockface 2

CAp_TNC_p1 = 1-(fv_ip1_TNC / (fv_ip1_TNC + v_ip1_TNC)) # TNC / PUDO curb accessibility on blockface 1
CAp_TNC_p2 = 1-(fv_ip2_TNC / (fv_ip2_TNC + v_ip2_TNC)) # TNC / PUDO curb accessibility on blockface 2
CAp_Cars_p1 = 1-(fv_ip1_Cars / (fv_ip1_Cars + v_ip1_Cars)) # Personal vehicle curb accessibility on blockface 1
CAp_Cars_p2 = 1-(fv_ip2_Cars / (fv_ip2_Cars + v_ip2_Cars)) # Personal vehicle curb accessibility on blockface 2


# Goods Curb Accessibility
CAg1 = 1-(fv_ig1 / (v_ig1 + fv_ig1)) # Goods curb accessibility on blockface 1
CAg2 = 1-(fv_ig2 / (v_ig2 + fv_ig2)) # Goods curb accessibility on blockface 2

# Total Accessibility
CAt1 = (1-((fv_ig1 + fv_ip1) / ((v_ig1 + v_ip1 + fv_ig1 + fv_ip1)))) # Total (all vehicle) curb accessibility on blockface 1
CAt2 = (1-((fv_ig2 + fv_ip2) / ((v_ig2 + v_ip2 + fv_ig2 + fv_ip2)))) # Total (all vehicle) curb accessibility on blockface 2

print(CAp1, CAp2)
print(CAp_TNC_p1, CAp_TNC_p2)
print(CAp_Cars_p1, CAp_Cars_p2)
print(CAg1, CAg2)
print(CAt1, CAt2)

1.0 1.0
1.0 1.0
1.0 1.0
0.6666666666666667 1.0
0.9787234042553191 1.0


# 5. Calculate space occupancy (for export to CSV)

In [22]:
#Remove unused columns from original dataframe
S01_02.drop(S01_02.columns[26:40],axis=1,inplace=True)
#S01_02.head(50)

In [23]:
# Get overall occupancy rate by dividing by simulations time (t)
occ_rate = (Park_S01_02.groupby(['DESTPARKLOT']).sum()[['DWELLTM']])/t

occ_rate_vehtype = (Park_S01_02.groupby(['DESTPARKLOT', 'VEHTYPE']).sum()[['DWELLTM']])/t

#occ_rate_vehtype

In [24]:
# the below calculate occupancy (% of simulation time) of each parking space 
# occupied by the major vehicle types (Personal (100), PUDO (640), and Goods vehicles (200,650))

PL_10_100 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==10.0) & (Park_S01_02['VEHTYPE']==100.0), 'DWELLTM'].sum())/t
PL_10_640 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==10.0) & (Park_S01_02['VEHTYPE']==640.0), 'DWELLTM'].sum())/t
PL_10_650 = ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==10.0) & (Park_S01_02['VEHTYPE']==650.0), 'DWELLTM'].sum())/t) + ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==10.0) & (Park_S01_02['VEHTYPE']==200.0), 'DWELLTM'].sum())/t)

PL_11_100 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==11.0) & (Park_S01_02['VEHTYPE']==100.0), 'DWELLTM'].sum())/t
PL_11_640 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==11.0) & (Park_S01_02['VEHTYPE']==640.0), 'DWELLTM'].sum())/t
PL_11_650 = ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==11.0) & (Park_S01_02['VEHTYPE']==650.0), 'DWELLTM'].sum())/t) + ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==11.0) & (Park_S01_02['VEHTYPE']==200.0), 'DWELLTM'].sum())/t)

PL_12_100 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==12.0) & (Park_S01_02['VEHTYPE']==100.0), 'DWELLTM'].sum())/t
PL_12_640 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==12.0) & (Park_S01_02['VEHTYPE']==640.0), 'DWELLTM'].sum())/t
PL_12_650 = ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==12.0) & (Park_S01_02['VEHTYPE']==650.0), 'DWELLTM'].sum())/t) + ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==12.0) & (Park_S01_02['VEHTYPE']==200.0), 'DWELLTM'].sum())/t)

PL_13_100 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==13.0) & (Park_S01_02['VEHTYPE']==100.0), 'DWELLTM'].sum())/t
PL_13_640 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==13.0) & (Park_S01_02['VEHTYPE']==640.0), 'DWELLTM'].sum())/t
PL_13_650 = ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==13.0) & (Park_S01_02['VEHTYPE']==650.0), 'DWELLTM'].sum())/t) + ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==13.0) & (Park_S01_02['VEHTYPE']==200.0), 'DWELLTM'].sum())/t)

PL_14_100 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==14.0) & (Park_S01_02['VEHTYPE']==100.0), 'DWELLTM'].sum())/t
PL_14_640 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==14.0) & (Park_S01_02['VEHTYPE']==640.0), 'DWELLTM'].sum())/t
PL_14_650 = ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==14.0) & (Park_S01_02['VEHTYPE']==650.0), 'DWELLTM'].sum())/t) + ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==14.0) & (Park_S01_02['VEHTYPE']==200.0), 'DWELLTM'].sum())/t)

PL_15_100 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==15.0) & (Park_S01_02['VEHTYPE']==100.0), 'DWELLTM'].sum())/t
PL_15_640 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==15.0) & (Park_S01_02['VEHTYPE']==640.0), 'DWELLTM'].sum())/t
PL_15_650 = ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==15.0) & (Park_S01_02['VEHTYPE']==650.0), 'DWELLTM'].sum())/t) + ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==15.0) & (Park_S01_02['VEHTYPE']==200.0), 'DWELLTM'].sum())/t)

PL_16_100 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==16.0) & (Park_S01_02['VEHTYPE']==100.0), 'DWELLTM'].sum())/t
PL_16_640 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==16.0) & (Park_S01_02['VEHTYPE']==640.0), 'DWELLTM'].sum())/t
PL_16_650 = ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==16.0) & (Park_S01_02['VEHTYPE']==650.0), 'DWELLTM'].sum())/t) + ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==16.0) & (Park_S01_02['VEHTYPE']==200.0), 'DWELLTM'].sum())/t)

PL_17_100 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==17.0) & (Park_S01_02['VEHTYPE']==100.0), 'DWELLTM'].sum())/t
PL_17_640 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==17.0) & (Park_S01_02['VEHTYPE']==640.0), 'DWELLTM'].sum())/t
PL_17_650 = ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==17.0) & (Park_S01_02['VEHTYPE']==650.0), 'DWELLTM'].sum())/t) + ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==17.0) & (Park_S01_02['VEHTYPE']==200.0), 'DWELLTM'].sum())/t)

PL_18_100 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==18.0) & (Park_S01_02['VEHTYPE']==100.0), 'DWELLTM'].sum())/t
PL_18_640 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==18.0) & (Park_S01_02['VEHTYPE']==640.0), 'DWELLTM'].sum())/t
PL_18_650 = ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==18.0) & (Park_S01_02['VEHTYPE']==650.0), 'DWELLTM'].sum())/t) + ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==18.0) & (Park_S01_02['VEHTYPE']==200.0), 'DWELLTM'].sum())/t)

PL_19_100 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==19.0) & (Park_S01_02['VEHTYPE']==100.0), 'DWELLTM'].sum())/t
PL_19_640 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==19.0) & (Park_S01_02['VEHTYPE']==640.0), 'DWELLTM'].sum())/t
PL_19_650 = ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==19.0) & (Park_S01_02['VEHTYPE']==650.0), 'DWELLTM'].sum())/t) + ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==19.0) & (Park_S01_02['VEHTYPE']==200.0), 'DWELLTM'].sum())/t)

PL_20_100 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==20.0) & (Park_S01_02['VEHTYPE']==100.0), 'DWELLTM'].sum())/t
PL_20_640 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==20.0) & (Park_S01_02['VEHTYPE']==640.0), 'DWELLTM'].sum())/t
PL_20_650 = ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==20.0) & (Park_S01_02['VEHTYPE']==650.0), 'DWELLTM'].sum())/t) + ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==20.0) & (Park_S01_02['VEHTYPE']==200.0), 'DWELLTM'].sum())/t)

PL_21_100 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==21.0) & (Park_S01_02['VEHTYPE']==100.0), 'DWELLTM'].sum())/t
PL_21_640 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==21.0) & (Park_S01_02['VEHTYPE']==640.0), 'DWELLTM'].sum())/t
PL_21_650 = ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==21.0) & (Park_S01_02['VEHTYPE']==650.0), 'DWELLTM'].sum())/t) + ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==21.0) & (Park_S01_02['VEHTYPE']==200.0), 'DWELLTM'].sum())/t)

PL_22_100 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==22.0) & (Park_S01_02['VEHTYPE']==100.0), 'DWELLTM'].sum())/t
PL_22_640 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==22.0) & (Park_S01_02['VEHTYPE']==640.0), 'DWELLTM'].sum())/t
PL_22_650 = ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==22.0) & (Park_S01_02['VEHTYPE']==650.0), 'DWELLTM'].sum())/t) + ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==22.0) & (Park_S01_02['VEHTYPE']==200.0), 'DWELLTM'].sum())/t)

PL_23_100 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==23.0) & (Park_S01_02['VEHTYPE']==100.0), 'DWELLTM'].sum())/t
PL_23_640 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==23.0) & (Park_S01_02['VEHTYPE']==640.0), 'DWELLTM'].sum())/t
PL_23_650 = ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==23.0) & (Park_S01_02['VEHTYPE']==650.0), 'DWELLTM'].sum())/t) + ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==23.0) & (Park_S01_02['VEHTYPE']==200.0), 'DWELLTM'].sum())/t)

PL_24_100 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==24.0) & (Park_S01_02['VEHTYPE']==100.0), 'DWELLTM'].sum())/t
PL_24_640 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==24.0) & (Park_S01_02['VEHTYPE']==640.0), 'DWELLTM'].sum())/t
PL_24_650 = ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==24.0) & (Park_S01_02['VEHTYPE']==650.0), 'DWELLTM'].sum())/t) + ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==24.0) & (Park_S01_02['VEHTYPE']==200.0), 'DWELLTM'].sum())/t)

PL_25_100 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==25.0) & (Park_S01_02['VEHTYPE']==100.0), 'DWELLTM'].sum())/t
PL_25_640 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==25.0) & (Park_S01_02['VEHTYPE']==640.0), 'DWELLTM'].sum())/t
PL_25_650 = ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==25.0) & (Park_S01_02['VEHTYPE']==650.0), 'DWELLTM'].sum())/t) + ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==25.0) & (Park_S01_02['VEHTYPE']==200.0), 'DWELLTM'].sum())/t)

PL_26_100 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==26.0) & (Park_S01_02['VEHTYPE']==100.0), 'DWELLTM'].sum())/t
PL_26_640 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==26.0) & (Park_S01_02['VEHTYPE']==640.0), 'DWELLTM'].sum())/t
PL_26_650 = ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==26.0) & (Park_S01_02['VEHTYPE']==650.0), 'DWELLTM'].sum())/t) + ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==26.0) & (Park_S01_02['VEHTYPE']==200.0), 'DWELLTM'].sum())/t)

PL_27_100 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==27.0) & (Park_S01_02['VEHTYPE']==100.0), 'DWELLTM'].sum())/t
PL_27_640 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==27.0) & (Park_S01_02['VEHTYPE']==640.0), 'DWELLTM'].sum())/t
PL_27_650 = ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==27.0) & (Park_S01_02['VEHTYPE']==650.0), 'DWELLTM'].sum())/t) + ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==27.0) & (Park_S01_02['VEHTYPE']==200.0), 'DWELLTM'].sum())/t)

PL_28_100 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==28.0) & (Park_S01_02['VEHTYPE']==100.0), 'DWELLTM'].sum())/t
PL_28_640 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==28.0) & (Park_S01_02['VEHTYPE']==640.0), 'DWELLTM'].sum())/t
PL_28_650 = ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==28.0) & (Park_S01_02['VEHTYPE']==650.0), 'DWELLTM'].sum())/t) + ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==28.0) & (Park_S01_02['VEHTYPE']==200.0), 'DWELLTM'].sum())/t)

# For Scenario 6 specifically (These spaces replace the busstop on blockface 2)
PL_30_100 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==30.0) & (Park_S01_02['VEHTYPE']==100.0), 'DWELLTM'].sum())/t
PL_30_640 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==30.0) & (Park_S01_02['VEHTYPE']==640.0), 'DWELLTM'].sum())/t
PL_30_650 = ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==30.0) & (Park_S01_02['VEHTYPE']==650.0), 'DWELLTM'].sum())/t) + ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==30.0) & (Park_S01_02['VEHTYPE']==200.0), 'DWELLTM'].sum())/t)

PL_31_100 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==31.0) & (Park_S01_02['VEHTYPE']==100.0), 'DWELLTM'].sum())/t
PL_31_640 = (Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==31.0) & (Park_S01_02['VEHTYPE']==640.0), 'DWELLTM'].sum())/t
PL_31_650 = ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==31.0) & (Park_S01_02['VEHTYPE']==650.0), 'DWELLTM'].sum())/t) + ((Park_S01_02.loc[(Park_S01_02['DESTPARKLOT']==31.0) & (Park_S01_02['VEHTYPE']==200.0), 'DWELLTM'].sum())/t)


In [25]:
#Average dwell time by VEHTYPE (for export to CSV)
Mean_DT_100 = Park_S01_02.loc[(Park_S01_02['VEHTYPE']==100.0), 'DWELLTM'].mean()
Mean_DT_640 = Park_S01_02.loc[(Park_S01_02['VEHTYPE']==640.0), 'DWELLTM'].mean()
Mean_DT_650 = Park_S01_02.loc[(Park_S01_02['VEHTYPE']==650.0), 'DWELLTM'].mean()

#print(Mean_DT_100,Mean_DT_640,Mean_DT_650)

## 6. Export processed data to .csv.

In [26]:
# Pre-requisite - Import the DictWriter class from csv  module
from csv import DictWriter
  
# The list of column names as mentioned in the CSV file
headersCSV = ['EXP', 'FILE', 'SCEN', 'CPI_P1', 'CPI_G1', 'CAp1', 'CAg1', 'CAt1', 'CPI_P2', 'CPI_G2', 'CAp2', 'CAg2', 'CAt2', 'Pax1', 'PaxVeh1', 'Goods1', 'GoodsVeh1', 'fv_ip1','fv_ig1', 'Pax2', 'PaxVeh2', 'Goods2', 'GoodsVeh2', 'fv_ip2', 'fv_ig2',
             'Mean_DT_100', 'Mean_DT_640', 'Mean_DT_650',
              'PL_10_100', 'PL_10_640', 'PL_10_650', 'PL_11_100', 'PL_11_640', 'PL_11_650', 'PL_12_100', 'PL_12_640', 'PL_12_650',
              'PL_13_100', 'PL_13_640', 'PL_13_650', 'PL_14_100', 'PL_14_640', 'PL_14_650', 'PL_15_100', 'PL_15_640', 'PL_15_650', 
              'PL_16_100', 'PL_16_640', 'PL_16_650', 'PL_17_100', 'PL_17_640', 'PL_17_650', 'PL_18_100', 'PL_18_640', 'PL_18_650', 
              'PL_19_100', 'PL_19_640', 'PL_19_650', 'PL_20_100', 'PL_20_640', 'PL_20_650', 'PL_21_100', 'PL_21_640', 'PL_21_650', 
              'PL_22_100', 'PL_22_640', 'PL_22_650', 'PL_23_100', 'PL_23_640', 'PL_23_650', 'PL_24_100', 'PL_24_640', 'PL_24_650', 
              'PL_25_100', 'PL_25_640', 'PL_25_650', 'PL_26_100', 'PL_26_640', 'PL_26_650', 'PL_27_100', 'PL_27_640', 'PL_27_650', 
              'PL_28_100', 'PL_28_640', 'PL_28_650', 'PL_30_100', 'PL_30_640', 'PL_30_650', 'PL_31_100', 'PL_31_640', 'PL_31_650',
              'Bus1', 'Bus_Pax1', 'Bus2', 'Bus_Pax2', 'v_ip1_TNC', 'v_ip2_TNC', 'v_ip1_Cars', 'v_ip2_Cars',
              'fv_ip1_TNC', 'fv_ip2_TNC', 'fv_ip1_Cars', 'fv_ip2_Cars', 'p_i1_TNC', 'p_i2_TNC', 'p_i1_Cars', 'p_i2_Cars',
              'CPI_TNC_p1', 'CPI_TNC_p2', 'CPI_Cars_p1', 'CPI_Cars_p2', 'CPI_Bus_p1', 'CPI_Bus_p2',
              'CAp_TNC_p1', 'CAp_TNC_p2', 'CAp_Cars_p1', 'CAp_Cars_p2'
             ]      
# The data assigned to the dictionary
# update every time: 'EXP', 'FILE', 'SCEN'
dict={'EXP':EX, 'FILE':filename, 'SCEN':SCEN, 'CPI_P1': CPI_p1, 'CPI_G1': CPI_g1, 'CAp1': CAp1, 'CAg1': CAg1, 'CAt1': CAt1, 'CPI_P2': CPI_p2, 'CPI_G2': CPI_g2, 'CAp2': CAp2, 'CAg2': CAg2, 'CAt2': CAt2, 'Pax1': p_i1, 'PaxVeh1': v_ip1, 'Goods1': g_i1, 'GoodsVeh1': v_ig1, 'fv_ip1':fv_ip1,'fv_ig1':fv_ig1, 'Pax2': p_i2, 'PaxVeh2': v_ip2, 'Goods2': g_i2, 'GoodsVeh2': v_ig2, 'fv_ip2':fv_ip2, 'fv_ig2':fv_ig2,
     'Mean_DT_100':Mean_DT_100, 'Mean_DT_640':Mean_DT_640, 'Mean_DT_650':Mean_DT_650,
      'PL_10_100':PL_10_100, 'PL_10_640':PL_10_640, 'PL_10_650':PL_10_650, 'PL_11_100':PL_11_100, 'PL_11_640':PL_11_640, 'PL_11_650':PL_11_650, 
      'PL_12_100':PL_12_100, 'PL_12_640':PL_12_640, 'PL_12_650':PL_12_650, 'PL_13_100':PL_13_100, 'PL_13_640':PL_13_640, 'PL_13_650':PL_13_650, 
      'PL_14_100':PL_14_100, 'PL_14_640':PL_14_640, 'PL_14_650':PL_14_650, 'PL_15_100':PL_15_100, 'PL_15_640':PL_15_640, 'PL_15_650':PL_15_650, 
      'PL_16_100':PL_16_100, 'PL_16_640':PL_16_640, 'PL_16_650':PL_16_650, 'PL_17_100':PL_17_100, 'PL_17_640':PL_17_640, 'PL_17_650':PL_17_650, 
      'PL_18_100':PL_18_100, 'PL_18_640':PL_18_640, 'PL_18_650':PL_18_650, 'PL_19_100':PL_19_100, 'PL_19_640':PL_19_640, 'PL_19_650':PL_19_650,
      'PL_20_100':PL_20_100, 'PL_20_640':PL_20_640, 'PL_20_650':PL_20_650, 'PL_21_100':PL_21_100, 'PL_21_640':PL_21_640, 'PL_21_650':PL_21_650, 
      'PL_22_100':PL_22_100, 'PL_22_640':PL_22_640, 'PL_22_650':PL_22_650, 'PL_23_100':PL_23_100, 'PL_23_640':PL_23_640, 'PL_23_650':PL_23_650, 
      'PL_24_100':PL_24_100, 'PL_24_640':PL_24_640, 'PL_24_650':PL_24_650, 'PL_25_100':PL_25_100, 'PL_25_640':PL_25_640, 'PL_25_650':PL_25_650,
      'PL_26_100':PL_26_100, 'PL_26_640':PL_26_640, 'PL_26_650':PL_26_650, 'PL_27_100':PL_27_100, 'PL_27_640':PL_27_640, 'PL_27_650':PL_27_650, 
      'PL_28_100':PL_28_100, 'PL_28_640':PL_28_640, 'PL_28_650':PL_28_650, 'PL_30_100':PL_30_100, 'PL_30_640':PL_30_640, 'PL_30_650':PL_30_650,      
      'PL_31_100':PL_31_100, 'PL_31_640':PL_31_640, 'PL_31_650':PL_31_650, 'Bus1':Bus1, 'Bus_Pax1':Bus_Pax1, 'Bus2':Bus2, 'Bus_Pax2':Bus_Pax2,
      'v_ip1_TNC':v_ip1_TNC, 'v_ip2_TNC':v_ip2_TNC, 'v_ip1_Cars':v_ip1_Cars, 'v_ip2_Cars':v_ip2_Cars,
      'fv_ip1_TNC':fv_ip1_TNC, 'fv_ip2_TNC':fv_ip2_TNC, 'fv_ip1_Cars':fv_ip1_Cars, 'fv_ip2_Cars':fv_ip2_Cars,
      'p_i1_TNC':p_i1_TNC, 'p_i2_TNC':p_i2_TNC, 'p_i1_Cars':p_i1_Cars, 'p_i2_Cars':p_i2_Cars,
      'CPI_TNC_p1':CPI_TNC_p1, 'CPI_TNC_p2':CPI_TNC_p2, 'CPI_Cars_p1':CPI_Cars_p1, 'CPI_Cars_p2':CPI_Cars_p2, 'CPI_Bus_p1':CPI_Bus_p1, 'CPI_Bus_p2':CPI_Bus_p2,
      'CAp_TNC_p1':CAp_TNC_p1, 'CAp_TNC_p2':CAp_TNC_p2, 'CAp_Cars_p1':CAp_Cars_p1, 'CAp_Cars_p2':CAp_Cars_p2      
     }
  
# Pre-requisite - The CSV file should be manually closed before running this code.

# First, open the old CSV file in append mode, hence mentioned as 'a'
# Then, for the CSV file, create a file object
with open('C://Users//type//your//directory//here//TypeFilenameHere.csv', 'a', newline='') as f_object:
    # Pass the CSV  file object to the Dictwriter() function
    # Result - a DictWriter object
    dictwriter_object = DictWriter(f_object, fieldnames=headersCSV)
    # Pass the data in the dictionary as an argument into the writerow() function
    dictwriter_object.writerow(dict)
    # Close the file object
    f_object.close()
    

In [27]:
print('done')

done
